In [1]:
import pandas as pd
import numpy as np
import os
import json

from classification_func import separate_meta_types, get_type_pivot

pd.options.display.max_colwidth = 500
pd.options.display.max_rows = 500

data_folder = '../Data/Event_identifiers'
data_files = os.listdir(data_folder)

# dictionary with events classification
with open('types_dict.json', 'r') as fp:
    types_dict = json.load(fp)

# Download events related to each convocation

In [2]:
df_events = None

for file in data_files:
    
    sep_symbol = ';' if '9' in file else ','
    
    df = pd.read_csv(os.path.join(data_folder, file), sep=sep_symbol)
    print(file, df.shape)
    
    if df_events is None:
        df_events = df
    else:
        df_events = pd.concat([df_events, df])
        
print(f'\nCombined dataset: {df_events.shape}')

# data sample
df_events.head().T

identifiers_9skl.csv (3857, 18)
identifiers_7skl.csv (4135, 18)
identifiers_8skl.csv (24103, 18)
identifiers_3skl.csv (7943, 18)
identifiers_4skl.csv (10929, 18)
identifiers_6skl.csv (23282, 18)
identifiers_5skl.csv (3424, 18)

Combined dataset: (77673, 18)


,0,1,2,3,4
date_agenda,16.04.2020,16.04.2020,16.04.2020,16.04.2020,16.04.2020
number|event,68038|5208,66285|5209,67740|5210,67742|5211,67742|5212
id_question,202004162,202004163,202004164,202004165,202004165
id_event,5208,5209,5210,5211,5212
event_name,"Поіменне голосування про проект Закону про ратифікацію Угоди про грант Фонду Е5Р (Проєкт ""Вища освіта України"") між Україною та Європейським інвестиційним банком (№0034) - в цілому",Поіменне голосування про проект Закону про внесення змін до Регламенту Верховної Ради України щодо протидії зловживанням прав народних депутатів у ході законодавчої процедури (№1043) - у другому читанні та в цілому,"Поіменне голосування про проект Закону про внесення змін до Закону України ""Про місцеве самоврядування в Україні"" (№2653) - у другому читанні та в цілому",Поіменне голосування про проект Закону про внесення змін до деяких законодавчих актів України (щодо посилення захисту телекомунікаційних мереж) (№2654) - за основу,Поіменне голосування про скорочення терміну підготовки до другого читання проекту Закону про внесення змін до деяких законодавчих актів України (щодо посилення захисту телекомунікаційних мереж) (№2654)
law_number,34,1043,2653,2654,2654
id,68038,66285,67740,67742,67742
mainExecutives,Комітет з питань інтеграції України з Європейським Союзом,"Комітет з питань Регламенту, депутатської етики та організації роботи Верховної Ради України","Комітет з питань організації державної влади, місцевого самоврядування, регіонального розвитку та містобудування",Комітет з питань правоохоронної діяльності,Комітет з питань правоохоронної діяльності
initiators,Президент України,"Фролов Павло Валерійович, Монастирський Денис Анатолійович, Совгиря Ольга Володимирівна, Клочко Андрій Андрійович, Юраш Святослав Андрійович, Костін Андрій Євгенович, Федієнко Олександр Павлович, Холодов Андрій Іванович, Буймістер Людмила Анатоліївна, Мотовиловець Андрій Вікторович, Бабак Сергій Віталійович, Арахамія Давид Георгійович, Культенко Артем Валерійович, Бардіна Марина Олегівна, Шуляк Олена Олексіївна, Безгін Віталій Юрійович, Василів Ігор Володимирович, Петруняк Євген Васильович, ...","Безгін Віталій Юрійович, Корнієнко Олександр Сергійович, Клочко Андрій Андрійович, Микиша Дмитро Сергійович, Шахов Сергій Володимирович, Юрчишин Петро Васильович, Шуляк Олена Олексіївна, Маріковський Олександр Валерійович, Пуртова Анна Анатоліївна, Качура Олександр Анатолійович, Гривко Сергій Дмитрович, Білозір Лариса Миколаївна, Загоруйко Аліна Леонідівна, Василів Ігор Володимирович, Ісаєнко Дмитро Валерійович, Кривошеєв Ігор Сергійович, Дунда Олег Андрійович, Підласа Роксолана Андріївна, М...","Федієнко Олександр Павлович, Ватрас Володимир Антонович, Захарченко Володимир Васильович, Неклюдов Владлен Михайлович, Медяник В'ячеслав Анатолійович, Яцик Юлія Григорівна, Соха Роман Васильович, Крячко Михайло Валерійович, Штепа Сергій Сергійович, Іонушас Сергій Костянтинович, Чернєв Єгор Володимирович, Мандзій Сергій Володимирович, Стефанчук Микола Олексійович","Федієнко Олександр Павлович, Ватрас Володимир Антонович, Захарченко Володимир Васильович, Неклюдов Владлен Михайлович, Медяник В'ячеслав Анатолійович, Яцик Юлія Григорівна, Соха Роман Васильович, Крячко Михайло Валерійович, Штепа Сергій Сергійович, Іонушас Сергій Костянтинович, Чернєв Єгор Володимирович, Мандзій Сергій Володимирович, Стефанчук Микола Олексійович"
registrationConvocation,IX скликання,IX скликання,IX скликання,IX скликання,IX скликання


# Data exploration
## what is the unique ID of an event?

In [3]:
print(f'Number of events: {len(df_events):,.0f}')
print(f'Number of unique event IDs: {len(df_events.id_event.unique()):,.0f}')
print(f'Number of unique IDs: {len(df_events.id.unique()):,.0f}')
print(f'Number of unique question IDs: {len(df_events.id_question.unique()):,.0f}')
print(f'Number of unique question number|event: {len(df_events["number|event"].unique()):,.0f}')

print('\n"number|event" column is a unique event identifier')

print('\nExample of events events with identical "id_event" values:')
display(df_events[df_events.id_event.isin([5184, 1305, 29505])].sort_values('id_event').T)

Number of events: 77,673
Number of unique event IDs: 73,232
Number of unique IDs: 15,804
Number of unique question IDs: 21,457
Number of unique question number|event: 77,673

"number|event" column is a unique event identifier

Example of events events with identical "id_event" values:


,2884,3422,18,19,3382,3383
date_agenda,29.10.2019,2006-05-25,13.04.2020,13.04.2020,2006-07-20,2006-07-20
number|event,66472|1305,00000|1305,67059|5184,68507|5184,00000|29505,27246|29505
id_question,2019102912,200605258,2020041314,2020041314,200607204,200607204
id_event,1305,1305,5184,5184,29505,29505
event_name,Поіменне голосування про розгляд за скороченою процедурою проекту Закону про внесення змін до статті 66 Кримінального процесуального кодексу України (щодо забезпечення права свідка на правову допомогу у кримінальному провадженні) (№1164),Поіменне голосування про пропозицію трьох фракцій щодо оголошення перерви у роботі Верховної Ради України до 7 червня 2006 року,Поіменне голосування про проект Постанови про скасування рішення Верховної Ради України від 30 березня 2020 року про прийняття в цілому як закону України проекту Закону про внесення змін до деяких законодавчих актів України щодо обігу земель сільськогосподарського призначення (реєстр. № 2178-10 від 10.10.2019) (№2178-10-П2) - в цілому,Поіменне голосування про проект Постанови про скасування рішення Верховної Ради України від 30 березня 2020 року про прийняття в цілому як закону України проекту Закону про внесення змін до деяких законодавчих актів України щодо обігу земель сільськогосподарського призначення (реєстр. № 2178-10 від 10.10.2019) (№2178-10-П2) - в цілому,Поіменне голосування про проект Закону про приєднання України до Конвенції про стягнення аліментів за кордоном (вiд 16.05.2006 № 1-1/475 вручено 07.06.2006) (№0001) - в цілому,Поіменне голосування про проект Закону про приєднання України до Конвенції про стягнення аліментів за кордоном (вiд 16.05.2006 № 1-1/475 вручено 07.06.2006) (№0001) - в цілому
law_number,1164,$,2178-10,2178-10-П2,$,0001
id,66472,0,67059,68507,0,27246
mainExecutives,Комітет з питань правоохоронної діяльності,NaN,Комітет з питань аграрної та земельної політики,"Комітет з питань Регламенту, депутатської етики та організації роботи Верховної Ради України",NaN,Комітет у закордонних справах
initiators,Шпенов Дмитро Юрійович,NaN,"Сольський Микола Тарасович, Нікітіна Марина Вікторівна, Нагаєвський Артем Сергійович, Богданець Андрій Володимирович, Шол Маргарита Віталіївна, Гайду Олександр Васильович, Кириченко Микола Олександрович, Тимофійчук Володимир Ярославович, Чорноморов Артем Олегович, Гузенко Максим Васильович, Бунін Сергій Валерійович, Костюк Дмитро Сергійович, Салійчук Олександр В'ячеславович, Литвиненко Сергій Анатолійович","Тимошенко Юлія Володимирівна, Івченко Вадим Євгенович",NaN,Президент України
registrationConvocation,IX скликання,NaN,IX скликання,IX скликання,NaN,V скликання


## Main categories of documents in the dataset

In [4]:
print('Column "rubric"')
df_events.loc[:, 'rubric'] = np.where(df_events.rubric.isna(), 'Undefined', df_events.rubric)
display(df_events.groupby('rubric')['number|event'].count().sort_values(ascending=False))

print('\nColumn "type"')
df_events.loc[:, 'type'] = np.where(df_events.type.isna(), 'Undefined', df_events.type)
display(df_events.groupby('type')['number|event'].count().sort_values(ascending=False))

print('\nColumns "type" and "registrationConvocation"')
df_events.loc[:, 'registrationConvocation'] = np.where(df_events.registrationConvocation.isna(), 'Undefined', df_events.registrationConvocation)
df_type_convocation = pd.pivot_table(data = df_events,
               index = "type",
               columns = "registrationConvocation",
               values = "number|event",
               aggfunc = "count",
               fill_value = 0)

df_type_convocation = df_type_convocation[['Undefined', 'III скликання', 'IV скликання', 'V скликання', 'VI скликання',
                     'VII скликання', 'VIII скликання', 'IX скликання']]

display(df_type_convocation)

Column "rubric"


rubric
Undefined                          16823
Галузевий розвиток                 14154
Економічна політика                12199
Правова політика                   10089
Державне будівництво                9503
Установчі повноваження              4136
Соціальна політика                  3073
Організаційні питання               2293
Безпека і оборона                   1791
Гуманітарна політика                1623
Двосторонні міжнародні угоди         945
Багатосторонні міжнародні угоди      544
Інші (заяви, звернення ВРУ)          484
Міжнародні угоди                      16
Name: number|event, dtype: int64


Column "type"


type
Проект Закону                      38785
Undefined                          16823
Проект Постанови                   10619
Проект                              5996
Пропозиції Президента до Закону     5437
Проект Заяви                           9
Проект Декларації                      3
Проект Звернення                       1
Name: number|event, dtype: int64


Columns "type" and "registrationConvocation"


registrationConvocation,Undefined,III скликання,IV скликання,V скликання,VI скликання,VII скликання,VIII скликання,IX скликання
type,,,,,,,,
Undefined,16823,0,0,0,0,0,0,0
Проект,0,6,50,0,1222,6,4712,0
Проект Декларації,0,0,0,0,2,1,0,0
Проект Закону,0,675,4254,1174,10855,2789,15739,3299
Проект Заяви,0,0,0,0,1,6,2,0
Проект Звернення,0,0,0,0,0,1,0,0
Проект Постанови,0,330,1869,947,3787,1175,2247,264
Пропозиції Президента до Закону,0,213,1263,698,2493,124,609,37


# Classify events

In [5]:
events_number = df_events.shape[0]
df_remainer, df_combined = separate_meta_types(df_events, types_dict)
events_classified = df_combined.shape[0]

print(f'Number of events in the database: {events_number:,.0f}')
print(f'Number of classified events: {events_classified:,.0f} ({events_classified/events_number*100:,.2f})')

# combine classified and not classified
df_remainer.loc[:, 'type_name'] = 'not classified'
df_remainer.loc[:, 'meta_type_name'] = 'not classified'
df_combined = pd.concat([df_combined, df_remainer])

Number of events in the database: 77,673
Number of classified events: 61,813 (79.58)


In [11]:
df_meta_convocation = get_type_pivot(df_combined, "registrationConvocation")
df_meta_type = get_type_pivot(df_combined, "type")
df_classification_count = df_combined.groupby(['meta_type_name', 'type_name'])[['number|event']].count()

display(df_meta_convocation)
print('\n\n')
display(df_meta_type)
print('\n\n')
display(df_classification_count)

df_meta_convocation.to_csv('summary_convocation.csv', index=False, sep="\t")
df_meta_type.to_csv('summary_document_type.csv', index=False, sep="\t")
df_classification_count.to_csv('summary_classification.csv', index=True, sep="\t")

,meta_type_name,Undefined,III скликання,IV скликання,V скликання,VI скликання,VII скликання,VIII скликання,IX скликання,Total,"Total, %"
0,поправки,2352.000,42.0000,1088.0000,459.0000,6212.0000,490.000,11031.0000,2399.0000,24073.0000,30.9928
1,"за основу, в цілому",1905.000,306.0000,3519.0000,1343.0000,6082.0000,1561.000,3700.0000,573.0000,18989.0000,24.4474
2,not classified,11102.000,876.0000,1602.0000,249.0000,1332.0000,275.000,408.0000,16.0000,15860.0000,20.4189
3,порядок денний,411.000,0.0000,577.0000,408.0000,1455.0000,839.000,2798.0000,107.0000,6595.0000,8.4907
4,скорочена процедура,409.000,0.0000,0.0000,47.0000,2011.0000,525.000,2501.0000,395.0000,5888.0000,7.5805
5,"Розгляд: повернення, зняття, перенесення, послідовність, відкликання, регламент розгляду",117.000,0.0000,259.0000,197.0000,549.0000,238.000,1325.0000,71.0000,2756.0000,3.5482
6,"сигнальне, рейтингове голосування",322.000,0.0000,0.0000,0.0000,46.0000,25.000,964.0000,0.0000,1357.0000,1.7471
7,повторне читання,58.000,0.0000,315.0000,22.0000,396.0000,97.000,352.0000,12.0000,1252.0000,1.6119
8,доопрацювання,71.000,0.0000,1.0000,75.0000,209.0000,25.000,210.0000,11.0000,602.0000,0.7750
9,Президент,8.000,0.0000,75.0000,19.0000,68.0000,27.000,20.0000,0.0000,217.0000,0.2794


,meta_type_name,Undefined,Проект,Проект Декларації,Проект Закону,Проект Заяви,Проект Звернення,Проект Постанови,Пропозиції Президента до Закону,Total,"Total, %"
0,поправки,2352.000,5511.0000,0.0,13468.000,0.0,0.0,282.0000,2460.0000,24073.0000,30.9928
1,"за основу, в цілому",1905.000,44.0000,2.0,9772.000,4.0,1.0,6102.0000,1159.0000,18989.0000,24.4474
2,not classified,11102.000,406.0000,0.0,1309.000,0.0,0.0,2156.0000,887.0000,15860.0000,20.4189
3,порядок денний,411.000,15.0000,0.0,4983.000,2.0,0.0,1005.0000,179.0000,6595.0000,8.4907
4,скорочена процедура,409.000,3.0000,1.0,4474.000,2.0,0.0,749.0000,250.0000,5888.0000,7.5805
5,"Розгляд: повернення, зняття, перенесення, послідовність, відкликання, регламент розгляду",117.000,5.0000,0.0,2366.000,0.0,0.0,123.0000,145.0000,2756.0000,3.5482
6,"сигнальне, рейтингове голосування",322.000,3.0000,0.0,814.000,0.0,0.0,145.0000,73.0000,1357.0000,1.7471
7,повторне читання,58.000,9.0000,0.0,1114.000,1.0,0.0,28.0000,42.0000,1252.0000,1.6119
8,доопрацювання,71.000,0.0000,0.0,450.000,0.0,0.0,29.0000,52.0000,602.0000,0.7750
9,Президент,8.000,0.0000,0.0,19.000,0.0,0.0,0.0000,190.0000,217.0000,0.2794


number|event
meta_type_name                                                                           type_name                                                                                                                     
not classified                                                                           not classified                                                                                                           15860
Президент                                                                                Поіменне голосування про подолання вето Президента України                                                                 199
                                                                                         Поіменне голосування про пропозиції Президента України                                                                       9
                                                                                         Поіменне голосування про підтримку пропозицій Президента України                                                             9
Розгляд: повернення, зняття, перенесення, послідовність, відкликання, регламент розгляду Поіменне голосування про відкликання проекту Закону                                                                         20
                                                                                         Поіменне голосування про відкликання суб'єктом права законодавчої ініціативи                                                 4
                                                                                         Поіменне голосування про відхилення проекту Закону                                                                          36
                                                                                         Поіменне голосування про відхилення проектів законів                                                                         2
                                                                                         Поіменне голосування про зняття з розгляду                                                                                 145
                                                                                         Поіменне голосування про переголосування питання щодо перенесення розгляду проекту Закону                                    1
                                                                                         Поіменне голосування про переголосування проекту Закону                                                                     28
                                                                                         Поіменне голосування про перенесення розгляду                                                                              398
                                                                                         Поіменне голосування про перенсення розгляду проектів законів                                                                4
                                                                                         Поіменне голосування про повернення до голосування проекту Закону                                                           38
                                                                                         Поіменне голосування про повернення до завершення процедури розгляду проектів законів                                        2
                                                                                         Поіменне голосування про повернення до переголосування                                                                     102
                                                                                         Поіменне голосування про повернення до розгляду проекту Закону                                                            1500
                                                                                         Поіменне г

In [13]:
df_combined.to_csv(os.path.join(data_folder, 'identifiers_combined.csv'), index=False, sep="\t")